In [1]:
import sys  
import os
import pandas as pd
import io
sys.path.insert(0, '../../../../scripts/')
sys.path.insert(0, '../../../../dataset/')
import hate_column_converter
import embedding_reader
import custom_keras_metrics
import tensorflow as tf
import keras.backend as K
import os
import time
import gc
import glob
import wget
import zipfile
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM
from tensorflow.keras.preprocessing import text, sequence
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from tensorflow.keras.optimizers import Adam
from keras.models import Model
from keras import backend as K
from tensorflow.keras.layers import Layer
from keras import initializers, regularizers, constraints, optimizers, layers
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras.layers import *
from keras.models import *
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.initializers import *
from keras.optimizers import *
from keras.callbacks import *
from keras import metrics
from tqdm import tqdm

2023-10-24 11:51:47.734972: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-10-24 11:51:47.735008: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
#wget.download("http://143.107.183.175:22980/download.php?file=embeddings/glove/glove_s300.zip", out="../../../../embeddings/")

In [3]:
#with zipfile.ZipFile("../../../../embeddings/glove_s300.zip", 'r') as zip_ref:
    #zip_ref.extractall("../../../../embeddings/")

In [2]:
MAX_LENGTH = 300
dataframe = pd.read_csv('../../../../dataset/HateBR.csv')
new_dataframe = hate_column_converter.convert_to_binary(dataframe)
classes_zero = new_dataframe[new_dataframe['hate_speech'] == 0]
classes_one = new_dataframe[new_dataframe['hate_speech'] == 1]
classes_zero_undersampled = classes_zero.sample(len(classes_one))
dataframe_undersampled = pd.concat([classes_zero_undersampled, classes_one]) 
instagram_comments = dataframe_undersampled['instagram_comments']
y = dataframe_undersampled['hate_speech']

/workspaces/hate-speech-detection-python/train/notebooks-my-paper/lstm/hate_speech/../../../../scripts/hate_column_converter.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lines_with_two_numbers_in_hate_speech[['col1', 'col2']] = lines_with_two_numbers_in_hate_speech['hate_speech'].str.split(',', expand=True)
/workspaces/hate-speech-detection-python/train/notebooks-my-paper/lstm/hate_speech/../../../../scripts/hate_column_converter.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lines_with_two_numb

In [3]:
x = dataframe_undersampled['instagram_comments']

In [4]:
token = Tokenizer()
token.fit_on_texts(x)
sequences = token.texts_to_sequences(x)
x = pad_sequences(sequences,maxlen=MAX_LENGTH)
vocabulary_size = len(token.word_index)+1

In [5]:
embedding_vector = embedding_reader.read_glove_file()
embedding_matrix = np.zeros((vocabulary_size,MAX_LENGTH))
for word,i in tqdm(token.word_index.items()):
    embedding_value = embedding_vector.get(word)
    if embedding_value is not None:
        embedding_matrix[i] = embedding_value

100%|██████████| 4928/4928 [00:00<00:00, 470232.28it/s]


In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, test_size=0.2)

In [8]:
lstm_model = Sequential()
lstm_model.add(Embedding(vocabulary_size,MAX_LENGTH,weights=[embedding_matrix],input_length=x.shape[1],trainable=False))
lstm_model.add(Dropout(0.25))
lstm_model.add(LSTM(64))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(1))
lstm_model.add(Activation('sigmoid'))
lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'binary_accuracy', custom_keras_metrics.f1_score])

In [9]:
model = lstm_model.fit(x_train, y_train, epochs=15, batch_size=128, validation_data=(x_test, y_test))

Epoch 1/15


2023-10-24 11:56:49.808443: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 46080000 exceeds 10% of free system memory.
2023-10-24 11:56:49.837359: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 46080000 exceeds 10% of free system memory.
2023-10-24 11:56:49.926823: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 46080000 exceeds 10% of free system memory.
2023-10-24 11:56:50.086661: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 46080000 exceeds 10% of free system memory.


 1/10 [==>...........................] - ETA: 34s - loss: 0.7068 - accuracy: 0.6016 - binary_accuracy: 0.6016 - f1_score: 0.7018

2023-10-24 11:56:51.423955: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 46080000 exceeds 10% of free system memory.


10/10 [==============================] - 9s 530ms/step - loss: 0.6832 - accuracy: 0.5666 - binary_accuracy: 0.5666 - f1_score: 0.6002 - val_loss: 0.6546 - val_accuracy: 0.6426 - val_binary_accuracy: 0.6426 - val_f1_score: 0.6349
Epoch 2/15
10/10 [==============================] - 5s 475ms/step - loss: 0.6360 - accuracy: 0.6629 - binary_accuracy: 0.6629 - f1_score: 0.6646 - val_loss: 0.6163 - val_accuracy: 0.6804 - val_binary_accuracy: 0.6804 - val_f1_score: 0.6898
Epoch 3/15
10/10 [==============================] - 5s 476ms/step - loss: 0.5942 - accuracy: 0.6913 - binary_accuracy: 0.6913 - f1_score: 0.7226 - val_loss: 0.5638 - val_accuracy: 0.7148 - val_binary_accuracy: 0.7148 - val_f1_score: 0.7083
Epoch 4/15
10/10 [==============================] - 5s 467ms/step - loss: 0.5531 - accuracy: 0.7300 - binary_accuracy: 0.7300 - f1_score: 0.7278 - val_loss: 0.5142 - val_accuracy: 0.7663 - val_binary_accuracy: 0.7663 - val_f1_score: 0.7527
Epoch 5/15
10/10 [==============================] -

In [10]:
results = lstm_model.evaluate(x_test, y_test, batch_size=128)

for name, value in zip(lstm_model.metrics_names, results):
  print(name, ': ', value)
print()

3/3 [==============================] - 0s 108ms/step - loss: 0.4908 - accuracy: 0.7629 - binary_accuracy: 0.7629 - f1_score: 0.7820
loss :  0.49081966280937195
accuracy :  0.7628865838050842
binary_accuracy :  0.7628865838050842
f1_score :  0.7820364832878113



: 

In [12]:
#import joblib

#model_filename = '../../../models/generated-models-base-paper/offensive_lstm_embeddings_model.pkl'
#joblib.dump(model, model_filename)

: 